[Notion](https://www.notion.so/aspirina180mg/Soy-Henry-49afb64d6c0641cc80dbc0378dd2db34?pvs=4)

[Kick-Off](https://drive.google.com/file/d/1l1SM-UAw5Mdkr6lmQ9X5UI4cxU_FRg_D/view)

[Slides](https://docs.google.com/presentation/d/1kRrCZDL-ZgZnm4DQ62Ovelf92uP-s4b7cdtp3gkK_kU/edit#slide=id.g2420ab8e36f_0_208)

[Repositorio](https://github.com/soyHenry/PI_DA/tree/Full_Time)

[Rúbrica](https://docs.google.com/spreadsheets/d/e/2PACX-1vTy9Ome3iLIl40SZ2jciZLiwARr9MVXfo_Mud3vVCin9P4zroAw_oxVQVch6m4TKivmUjFrYJKMJJhS/pubhtml)


# **ETL**


## **Importación de librerías**


- `pandas` para realizar análisis de datos.
- `numpy` para realisar cálculos matemáticos y estadísticos.


In [1]:
import pandas as pd
import numpy as np

## **Carga de datos y creación de dataframes**


### **Se carga cada hoja del documento `homicidios.xlsx` a dataframes por separado.**


- HECHOS se carga en df_hechos
- VICTIMAS se carga en df_victimas

In [2]:
df_hechos = pd.read_excel("Datasets\homicidios.xlsx", sheet_name="HECHOS")
df_victimas = pd.read_excel("Datasets\homicidios.xlsx", sheet_name="VICTIMAS")

## **Se eliminan columnas redundantes o duplicadas entre `df_hechos` y `df_victimas`**


columnas eliminadas de df_hechos

- AAAA (información se encuentra en `df_hechos.FECHA`).
- MM (información se encuentra en `df_hechos.FECHA`).
- DD (información se encuentra en `df_hechos.FECHA`).
- HH (información se encuentra en `df_hechos.HORA`).
- df_hechos.XY (CABA) (información se encuentra en `pos x` y `pos y`).
- pos y (información se encuentra en `df_hechos.XY (CABA)`).
- PARTICIPANTES (información se encuentra en `df_hechos.ACUSADO` y en `df_victimas.VICTIMA`).
- VICTIMA (cuplicado de `df_victimas.VICTIMA`).
  <br><br>
  columnas eliminadas de df_victimas
- FECHA (duplicado de `df_hechos.FECHA`).
- AAAA (información se encuentra en `df_hechos.FECHA`).
- MM (información se encuentra en `df_hechos.FECHA`).
- DD (información se encuentra en `df_hechos.FECHA`).


In [3]:
df_hechos = df_hechos.drop(
    ["AAAA", "MM", "DD", "HH", "XY (CABA)", "PARTICIPANTES", "VICTIMA"], axis=1
)

df_victimas = df_victimas.drop(["FECHA", "AAAA", "MM", "DD"], axis=1)

## **Se crea el dataframe `df_homicidios` mezclando df_hechos y df_victimas a través de `df_hechos.ID` y `df_victimas.ID_Hecho`**


In [4]:
df_homicidios = pd.merge(df_hechos, df_victimas, left_on="ID", right_on="ID_hecho")
df_homicidios = df_homicidios[
    [
        "ID",
        "ACUSADO",
        "FECHA",
        "HORA",
        "COMUNA",
        "LUGAR_DEL_HECHO",
        "TIPO_DE_CALLE",
        "Calle",
        "Altura",
        "Cruce",
        "Dirección Normalizada",
        "pos x",
        "pos y",
        "N_VICTIMAS",
        "VICTIMA",
        "ROL",
        "SEXO",
        "EDAD",
        "FECHA_FALLECIMIENTO",
    ]
]
del df_hechos
del df_victimas

## **Se carga cada hoja del documento `lesiones.xlsx` a dataframes por separado.**


- HECHOS se carga en df_hechos
- VICTIMAS se carga en df_victimas


In [5]:
df_hechos = pd.read_excel("Datasets\lesiones.xlsx", sheet_name="HECHOS")
df_victimas = pd.read_excel("Datasets\lesiones.xlsx", sheet_name="VICTIMAS")

## **Se eliminan columnas redundantes o duplicadas entre `df_hechos` y `df_victimas`**


columnas eliminadas de df_hechos

- aaaa (información se encuentra en `df_hechos.FECHA`)
- mm (información se encuentra en `df_hechos.FECHA`)
- dd (información se encuentra en `df_hechos.FECHA`)
- hora (información se encuentra en `df_hechos.HORA`)
- geocodificacion_CABA (información se encuentra en `df_hechos.longitud` y `df_hechos.latitud`)
- victima (cuplicado de `df_victimas.VICTIMA`)
- participantes (información se encuentra en `df_hechos.ACUSADO` y en `df_victimas.VICTIMA`)

<br><br>
columnas eliminadas de df_victimas

- AAAA (información se encuentra en `df_hechos.FECHA`)
- MM (información se encuentra en `df_hechos.FECHA`)
- DD (información se encuentra en `df_hechos.FECHA`)
- FECHA (duplicado de `df_hechos.FECHA`)


In [6]:
df_hechos = df_hechos.drop(
    ["aaaa", "mm", "dd", "hora", "geocodificacion_CABA", "victima", "participantes"],
    axis=1,
)

df_victimas = df_victimas.drop(["AAA", "MM", "DD", "FECHA "], axis=1)

## **Se crea el dataframe `df_lesiones` mezclando df_hechos y df_victimas a través de `df_hechos.ID` y `df_victimas.ID_Hecho`**


In [7]:
df_lesiones = pd.merge(df_hechos, df_victimas, left_on="id", right_on="ID hecho")
df_lesiones = df_lesiones.rename(columns={'latutid': 'latitud'})
df_lesiones = df_lesiones[
    [
        "id",
        "acusado",
        "fecha",
        "franja_hora",
        "comuna",
        "otra_direccion",
        "tipo_calle",
        "calle",
        "altura",
        "cruce",
        "direccion_normalizada",
        "longitud",
        "latitud",
        "n_victimas",
        "VEHICULO_VICTIMA",
        "moto",
        "auto",
        "transporte_publico",
        "camion",
        "ciclista",
        "gravedad",
        "SEXO",
        "EDAD_VICTIMA",
        "GRAVEDAD",
    ]
]
del df_victimas
del df_hechos

## **Se reemplazan los datos con "SD" (Sin dato) por NaN**


In [8]:
df_homicidios.replace("SD", np.nan, inplace=True)
df_homicidios.infer_objects(copy=False)

df_lesiones.replace("SD", np.nan, inplace=True)
df_lesiones.infer_objects(copy=False)

C:\Users\misag\AppData\Local\Temp\ipykernel_18736\465287807.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_homicidios.replace("SD", np.nan, inplace=True)


,id,acusado,fecha,franja_hora,comuna,otra_direccion,tipo_calle,calle,altura,cruce,...,VEHICULO_VICTIMA,moto,auto,transporte_publico,camion,ciclista,gravedad,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000179,NaN,2019-01-01 00:00:00,9,14,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,NaN,...,sd,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN
1,LC-2019-0000053,NaN,2019-01-01 00:00:00,1,8,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,NaN,...,sd,NaN,x,NaN,NaN,NaN,NaN,Varon,57,NaN
2,LC-2019-0000063,NaN,2019-01-01 00:00:00,2,8,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,NaN,...,sd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LC-2019-0000079,NaN,2019-01-01 00:00:00,2,7,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,NaN,...,sd,x,NaN,NaN,NaN,NaN,NaN,Varon,NaN,NaN
4,LC-2019-0000082,NaN,2019-01-01 00:00:00,4,3,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,NaN,...,sd,NaN,NaN,x,NaN,NaN,NaN,Varon,45,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27593,LC-2021-0652865,NaN,2021-12-31 00:00:00,19,9,NaN,AVENIDA,"ALBERDI, JUAN BAUTISTA AV.",4436.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varon,28,NaN
27594,LC-2021-0652865,NaN,2021-12-31 00:00:00,19,9,NaN,AVENIDA,"ALBERDI, JUAN BAUTISTA AV.",4436.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varon,52,NaN
27595,LC-2021-0652907,NaN,2021-12-31 00:00:00,20,1,Barrio Padre Carlos Mugica (Villa 31 bis),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varon,23,NaN
27596,LC-2021-0652921,TRANSPORTE PUBLICO,2021-12-31 00:00:00,22,5,NaN,CALLE,LINIERS VIRREY,0.0,MORENO,...,MOTO,x,0,x,0,0,GRAVE,Varon,31,GRAVE


## **Eliminación de filas con datos faltantes o erroneos**


- Filas con datos faltantes en "ACUSADO", "VICTIMA" y "ROL" al mismo tiempo


In [9]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     717 non-null    object        
 1   ACUSADO                694 non-null    object        
 2   FECHA                  717 non-null    datetime64[ns]
 3   HORA                   716 non-null    object        
 4   COMUNA                 717 non-null    int64         
 5   LUGAR_DEL_HECHO        716 non-null    object        
 6   TIPO_DE_CALLE          717 non-null    object        
 7   Calle                  716 non-null    object        
 8   Altura                 133 non-null    float64       
 9   Cruce                  540 non-null    object        
 10  Dirección Normalizada  708 non-null    object        
 11  pos x                  717 non-null    object        
 12  pos y                  717 non-null    object        
 13  N_VIC

In [10]:
columnas_revisadas = ["ACUSADO", "VICTIMA", "ROL"]
filas_invalidas = df_homicidios[
    df_homicidios[columnas_revisadas].isnull().all(axis=1)].index
df_homicidios = df_homicidios.drop(filas_invalidas)



'''para trabajar'''
columnas_revisadas = ["VICTIMA"]
filas_invalidas = df_homicidios[df_homicidios[columnas_revisadas].isnull().all(axis=1)]
print(filas_invalidas)

            ID ACUSADO      FECHA      HORA  COMUNA  \
89   2016-0115  CARGAS 2016-09-02  11:50:00      12   
93   2016-0119     NaN 2016-09-04  06:40:00      12   
167  2017-0029    AUTO 2017-03-07  02:15:00       9   
221  2017-0089    AUTO 2017-07-13  12:05:00       9   
280  2017-0155    MOTO 2017-12-12  21:25:41       7   

                           LUGAR_DEL_HECHO TIPO_DE_CALLE  \
89                        ARIAS Y TRONADOR         CALLE   
93   CRISOLOGO LARRALDE Y AVDA TRIUNVIRATO       AVENIDA   
167                        MOZART Y TANDIL         CALLE   
221              MARTINIANO LEGUIZAMON 901         CALLE   
280                     AV. RIVADAVIA 6350       AVENIDA   

                       Calle  Altura            Cruce  \
89                     ARIAS     NaN         TRONADOR   
93   LARRALDE, CRISOLOGO AV.     NaN  TRIUNVIRATO AV.   
167                   MOZART     NaN           TANDIL   
221   LEGUIZAMON, MARTINIANO   901.0              NaN   
280            RIVADAVI

In [11]:
%who

columnas_revisadas	 df_homicidios	 df_lesiones	 filas_invalidas	 np	 pd	 


# Sigue haciendo drops, para dejar limpio el dataset

In [12]:
''' Intentar justificar este drop tan específico '''


df_homicidios = df_homicidios.drop(
    df_homicidios.loc[df_homicidios["ID"] == "2016-0151"].index
)

## **Revisión de tipo de datos por columna y conversión de tipo de datos**


### ver qué hacer con el siguiente código, al parecer me sirve para lo de tipo por columna

In [13]:
df_homicidios.head()

,ID,ACUSADO,FECHA,HORA,COMUNA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,pos x,pos y,N_VICTIMAS,VICTIMA,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,AUTO,2016-01-01,04:00:00,8,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",-58.47533969,-34.68757022,1,MOTO,CONDUCTOR,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,PASAJEROS,2016-01-02,01:15:00,9,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",-58.50877521,-34.66977709,1,AUTO,CONDUCTOR,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,AUTO,2016-01-03,07:00:00,1,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,-58.39040293,-34.63189362,1,MOTO,CONDUCTOR,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,NaN,2016-01-10,00:00:00,8,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",-58.46503904,-34.68092974,1,MOTO,CONDUCTOR,MASCULINO,18.0,NaN
4,2016-0005,PASAJEROS,2016-01-21,05:20:00,1,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",-58.38718297,-34.62246630,1,MOTO,CONDUCTOR,MASCULINO,29.0,2016-02-01 00:00:00


In [14]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 716
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     712 non-null    object        
 1   ACUSADO                694 non-null    object        
 2   FECHA                  712 non-null    datetime64[ns]
 3   HORA                   711 non-null    object        
 4   COMUNA                 712 non-null    int64         
 5   LUGAR_DEL_HECHO        712 non-null    object        
 6   TIPO_DE_CALLE          712 non-null    object        
 7   Calle                  712 non-null    object        
 8   Altura                 132 non-null    float64       
 9   Cruce                  539 non-null    object        
 10  Dirección Normalizada  704 non-null    object        
 11  pos x                  712 non-null    object        
 12  pos y                  712 non-null    object        
 13  N_VICTIMAS

In [15]:
# Convertir la columna "HORA" a tipo datetime con el formato "%H:%M:%S"
df_homicidios["HORA"] = pd.to_datetime(df_homicidios["HORA"], format="%H:%M:%S")
df_homicidios["FECHA_FALLECIMIENTO"] = pd.to_datetime(
    df_homicidios["FECHA_FALLECIMIENTO"]
)

---


# Salida a Parquet / Salida a SQL

In [30]:
''' # Código para Parquet
df_homicidios.to_parquet("Datasets/df_homicidios.parquet")
df_lesiones.to_parquet("Datasets/df_lesiones.parquet")''' 

df_homicidios['HORA'] = df_homicidios['HORA'].replace('SD', np.nan)
df_homicidios['HORA'] = pd.to_numeric(df_homicidios['HORA'], errors='coerce')
df_lesiones['franja_hora'] = df_lesiones['franja_hora'].replace('SD', np.nan)
df_lesiones['franja_hora'] = pd.to_numeric(df_lesiones['franja_hora'], errors='coerce')
df_homicidios['FECHA'] = pd.to_datetime(df_homicidios['FECHA'])
df_lesiones['fecha'] = pd.to_datetime(df_lesiones['fecha'])
df_lesiones['comuna'] = df_lesiones['comuna'].replace('No Especificada', np.nan)
df_lesiones['comuna'] = pd.to_numeric(df_lesiones['comuna'], errors='coerce')
df_lesiones['longitud'] = df_lesiones['longitud'].astype(str)
df_lesiones['latitud'] = df_lesiones['latitud'].astype(str)
df_lesiones['moto'] = df_lesiones['moto'].astype(str)
df_lesiones['auto'] = df_lesiones['auto'].astype(str)
df_lesiones['transporte_publico'] = df_lesiones['transporte_publico'].astype(str)
df_lesiones['camion'] = df_lesiones['camion'].astype(str)
df_lesiones['ciclista'] = df_lesiones['ciclista'].astype(str)
df_lesiones['EDAD_VICTIMA'] = df_lesiones['EDAD_VICTIMA'].fillna(0)
df_lesiones['EDAD_VICTIMA'] = pd.to_numeric(df_lesiones['EDAD_VICTIMA'], errors='coerce')
df_lesiones = df_lesiones[np.isfinite(df_lesiones['EDAD_VICTIMA'])]
df_lesiones['EDAD_VICTIMA'] = df_lesiones['EDAD_VICTIMA'].astype('int64')



# Guardar los DataFrames corregidos en formato Parquet
df_homicidios.to_parquet("Datasets/df_homicidios.parquet")
df_lesiones.to_parquet("Datasets/df_lesiones.parquet")



''' # Código para SQL

from sqlalchemy import create_engine
import pandas as pd

# Configuración de la conexión a la base de datos MySQL
nombre_usuario = 'root'
contrasena = 'Mamago12'
nombre_base_datos = 'STOP'
host = 'localhost'
puerto = '3306'  # Puerto predeterminado para MySQL

# Crear un engine de sqlalchemy para la conexión a la base de datos MySQL
engine = create_engine(f"mysql+pymysql://{nombre_usuario}:{contrasena}@{host}:{puerto}/{nombre_base_datos}")

# Exportar el DataFrame df_homicidios a una tabla en MySQL
nombre_tabla_homicidios = 'tabla_homicidios'
df_homicidios.to_sql(nombre_tabla_homicidios, con=engine, if_exists='append', index=False)

# Exportar el DataFrame df_lesiones a una tabla en MySQL
nombre_tabla_lesiones = 'tabla_lesiones'
df_lesiones.to_sql(nombre_tabla_lesiones, con=engine, if_exists='append', index=False)

# Confirmación de la exportación
print(f"DataFrames exportados y datos agregados a las tablas {nombre_tabla_homicidios} y {nombre_tabla_lesiones} en la base de datos MySQL.")
'''

' # Código para SQL\n\nfrom sqlalchemy import create_engine\nimport pandas as pd\n\n# Configuración de la conexión a la base de datos MySQL\nnombre_usuario = \'root\'\ncontrasena = \'Mamago12\'\nnombre_base_datos = \'STOP\'\nhost = \'localhost\'\npuerto = \'3306\'  # Puerto predeterminado para MySQL\n\n# Crear un engine de sqlalchemy para la conexión a la base de datos MySQL\nengine = create_engine(f"mysql+pymysql://{nombre_usuario}:{contrasena}@{host}:{puerto}/{nombre_base_datos}")\n\n# Exportar el DataFrame df_homicidios a una tabla en MySQL\nnombre_tabla_homicidios = \'tabla_homicidios\'\ndf_homicidios.to_sql(nombre_tabla_homicidios, con=engine, if_exists=\'append\', index=False)\n\n# Exportar el DataFrame df_lesiones a una tabla en MySQL\nnombre_tabla_lesiones = \'tabla_lesiones\'\ndf_lesiones.to_sql(nombre_tabla_lesiones, con=engine, if_exists=\'append\', index=False)\n\n# Confirmación de la exportación\nprint(f"DataFrames exportados y datos agregados a las tablas {nombre_tabla_ho

----

In [ ]:
print(df_homicidios.columns)

In [27]:
print(df_lesiones.columns)

Index(['id', 'acusado', 'fecha', 'franja_hora', 'comuna', 'otra_direccion',
       'tipo_calle', 'calle', 'altura', 'cruce', 'direccion_normalizada',
       'longitud', 'latitud', 'n_victimas', 'VEHICULO_VICTIMA', 'moto', 'auto',
       'transporte_publico', 'camion', 'ciclista', 'gravedad', 'SEXO',
       'EDAD_VICTIMA', 'GRAVEDAD', 'automoto'],
      dtype='object')


# **Enunciado Henry**

Las tasas de mortalidad relacionadas con siniestros viales suelen ser un indicador crítico de la seguridad vial en una región. Estas tasas se calculan, generalmente, como el número de muertes por cada cierto número de habitantes o por cada cierta cantidad de vehículos registrados. Reducir estas tasas es un objetivo clave para mejorar la seguridad vial y proteger la vida de las personas en la ciudad.

Es importante destacar que la prevención de siniestros viales involucra medidas como la educación vial, el cumplimiento de las normas de tráfico, la infraestructura segura de carreteras y calles, así como la promoción de vehículos más seguros. El seguimiento de las estadísticas y la implementación de políticas efectivas son esenciales para abordar este problema de manera adecuada.

## **Contexto**

En Argentina, cada año mueren cerca de 4.000 personas en siniestros viales. Aunque muchas jurisdicciones han logrado disminuir la cantidad de accidentes de tránsito, esta sigue siendo la principal causa de muertes violentas en el país.
Los informes del Sistema Nacional de Información Criminal (SNIC), del Ministerio de Seguridad de la Nación, revelan que entre 2018 y 2022 se registraron 19.630 muertes en siniestros viales en todo el país. Estas cifras equivalen a 11 personas por día que resultaron víctimas fatales por accidentes de tránsito.

Solo en 2022, se contabilizaron 3.828 muertes fatales en este tipo de hechos. Los expertos en la materia indican que en Argentina es dos o tres veces más alta la probabilidad de que una persona muera en un siniestro vial que en un hecho de inseguridad delictiva

## **Rol a desarrollar**

El `Observatorio de Movilidad y Seguridad Vial` (OMSV), centro de estudios que se encuentra bajo la órbita de la **_Secretaría de Transporte_** del Gobierno de la Ciudad Autónoma de Buenos Aires, nos solicita la elaboración de un proyecto de anális de datos, con el fin de generar información que le permita a las autoridades locales tomar medidas para disminuir la cantidad de víctimas fatales de los siniestros viales.
Para ello, nos disponibilizan un dataset sobre homicidios en siniestros viales acaecidos en la Ciudad de Buenos Aires durante el periodo 2016-2021. Este dataset se encuentra en formato _xlsx_ y contiene dos hojas llamadas: **hechos** y **víctimas**. Asimismo, observarán que incluye otras dos hojas adicionales de diccionarios de datos, que les podrá servir de guía para un mayor entendimiento de la data compartida.

# **Propuesta de trabajo -mínimos entregables-**

_Es importante que a la hora de empezar a desarrollar cada item, y tu demo, te ayudes tambien de la [rúbrica de evaluación](https://docs.google.com/spreadsheets/d/e/2PACX-1vTy9Ome3iLIl40SZ2jciZLiwARr9MVXfo_Mud3vVCin9P4zroAw_oxVQVch6m4TKivmUjFrYJKMJJhS/pubhtml)._ 😄

## `EDA` (Exploratory Data Analysis)

Debes realizar un análisis exploratorio de los datos en un notebook. Tienen que estar tus pasos documentados con claridad, con las conclusiones correspondientes en cada gráfico empleado y análisis de lo que vas observando, utilizando celdas Markdown para tal fin. La prolijidad del notebook será un aspecto a evaluar. Es importante que tengas en cuenta que, en muchas oportunidades y trabajos, un EDA constituye un entregable en sí mismo.

En esta línea, hay varios aspectos indispensables que **deben** ser abordados en cualquier Análisis Exploratorio de Datos y tomaremos como punto de partida para evaluar tu performance en este apartado. Entre estos aspectos destacados se encuentran: _búsqueda de valores faltantes, valores atípicos/extremos u outliers y registros duplicados_. Asimismo, la utilización de gráficos coherentes según la tipología de variable que corresponda resulta esencial.

**_En caso de hacer uso de librerías como pandas_profiling, es indispensable acompañar los gráficos con análisis propios._**

*   Revisar por Estacionalidad

*   [dale un vistazo al workshop de preprocesamiento](https://www.app.soyhenry.com/lecture/dc30f5d1-573e-4f65-a7c6-7058eabcbcf4)

## `Dashboard`

Debe ser funcional y coherente con el storytelling. El dasbhoard tiene que incluir **filtros**, permitiendo explorar detalladamente los datos con la selección de cada uno de ellos. Es decir, es indispensable que sea **interactivo**. También, se espera que el diseño que implementen facilite la interpretación de la información y su análisis, siendo importante, para ello, la claridad en la presentación de los datos, aspectos inherentes a la esteticidad, elección coherente de los gráficos según las variables a visualizar, entre otros ítems.
*   [Utiliza el workshop de streamlit para aprender](https://www.app.soyhenry.com/lecture/dc30f5d1-573e-4f65-a7c6-7058eabcbcf4)
*   [Debes ver el workshop de Data to Viz y Storytelling, ayudará a crear el dashboard](https://www.app.soyhenry.com/lecture/dc30f5d1-573e-4f65-a7c6-7058eabcbcf4)
*   El Dashboard refleja lo del eda, pero más boniti
*   Considera agregar botones de navegación, dan un plus
*   Hacerlo en modo obscuro(?) (será mejor hacerlo cambiable?)
*   KPi primero o después? qué quiero para mi Storytelling?

## `Análisis` :warning:

No se considerará solamente la producción de gráficos con datos -dashboard-, sino también los análisis y conclusiones que puedan extraer a partir de ellos.

## `KPIs`

Debes graficar y medir los 2 KPIs propuestos a continuación, representándolos adecuadamente en el dashboard. A su vez, tambíen tienes que proponer, medir y graficar un tercer KPI que consideres relevante para la temática.
Los dos KPIs propuestos son:

- _Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en Ciudad Autónoma de Buenos Aires (CABA), en comparación con la tasa de homicidios en siniestros viales del semestre anterior_.

  Definimos a la **tasa de homicidios en siniestros viales** como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico.
  Su fórmula es: (Número de homicidios en siniestros viales / Población total) \* 100,000

- _Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en Ciudad Autónoma de Buenos Aires (CABA), respecto al año anterior_.

  Definimos a la **cantidad de accidentes mortales de motociclistas en siniestros viales** como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal.
  Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) \* 100

`MUY IMPORTANTE` repasar qué es un KPI y cómo se diferencia de una métrica convencional. En el material de apoyo tienen lectura que puede ser de ayuda.</small>

## `Repositorio de GitHub`

El repositorio debe contener un **Readme** principal donde presenten, en una primera instancia, de forma general **su proyecto** y detallen qué hay en cada archivo/carpeta del propio repositorio. Este Readme no puede ser el mismo de la consigna que nosotros les entregamos.
A su vez, el Readme debe incluir un **reporte de análisis con base en sus dashboards**, así como el análisis y la funcionalidad de los KPIs sugeridos.

## _**Desafíate y no te quedes siendo Junior, sé Junior Advanced**_

Pensando en alcanzar tu Boom 🚀, te recomendamos incorporar los siguientes desafíos para tener un portfolio mucho más completo y competitivo:

- Crear una base de datos en un motor SQL, ingestar el dataset procesado y utilizarla como fuente de datos de su dashboard en Power BI (o la herramienta de visualización que utilice).
- Ejecutar scripts de Python en la herramienta de visualización de datos escogida.
- Cruce de datos con datasets complementarios, ya sea para obtener información nueva o poder comparar la información disponible para todas las plataformas.

<sub> Nota: la realización de uno o más de estos ítems no es intercambiable con los requerimientos mínimos establecidos en la sección anterior "Propuesta de trabajo". Empiece con esta sección una vez haya cumplido con los requerimientos mínimos, a modo de desafiarse a usted mismo y destacar frente al resto.</sub>

# Fuente de datos

## **Obligatorio:**

- [Buenos Aires Data](https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales): deberán utilizar el dataset denominado `Homicidios`

## **Complementarios:**

- [Buenos Aires Data](https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales): pueden usar el dataset de `Lesiones`
- Cualquier dataset de búsqueda propia que complemente y mejore el análisis. Recuerde el uso de [APIs y WebScrapping](https://www.students.soyhenry.com/classes/95?cohortId=124&videoOrdinal=2)

# Lo que tendremos en cuenta a la hora de evaluar

Serás evaluado en dos grandes áreas `Tech` y `Soft`!

Ambas con igual peso entre si y ambas deben ser aprobatorias para tener la calidad de aprobado en este PIDA. Ten presente que una nota minima para aprobar significa tener TODOS los items como "Bueno" 👌
A continuacion te facilitamos nuevamente la [rúbrica de evaluación](https://docs.google.com/spreadsheets/d/e/2PACX-1vTy9Ome3iLIl40SZ2jciZLiwARr9MVXfo_Mud3vVCin9P4zroAw_oxVQVch6m4TKivmUjFrYJKMJJhS/pubhtml) con la que seras evaluado por tu corrector@. Recuerda que el feedback de tu corrector@ no es en ningun momento un indicativo de tu nota. Si tienes alguna duda durante tu DEMO, pidele a tu corrector@ que te aclare claramente cuales son los objetivos de aprendizaje no cumplidos.

Esperamos que te sirva de guia de aprendizaje, y recuerda que no se trata solo de cumplir requisitos, sino de destacar en cada nivel 🚀 💛

## Material de apoyo

- [Notas para el uso del dataset de homicidios de siniestros viales de la CABA](https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/NOTAS_HOMICIDIOS_SINIESTRO_VIAL.pdf)
- [Observatorio de Movilidad y Seguridad Vial de la Ciudad Autónoma de Buenos Aires](https://buenosaires.gob.ar/observatorio-de-movilidad-y-seguridad-vial)

## Tech

- [Repaso de clase sobre EDA](https://www.students.soyhenry.com/classes/100?cohortId=106&videoOrdinal=1)
- [Code Review: **Interactividad** Dashboard, Patron Z, **Tooltips**](https://www.students.soyhenry.com/classes/93?cohortId=124&videoOrdinal=2)
- [KPI's 4 students](https://docs.google.com/document/d/1DI0ZVgHfOfjgnXGhi8jEKzwCIjtUdgRUDe-qiiGGq8E/edit)
- [Code Review: DAX y **medidas calculadas**](https://www.students.soyhenry.com/classes/96?cohortId=124&videoOrdinal=2)

## Soft

- ¡Todos los Workshops de esta etapa serán de gran utilidad para tener un proyecto exitoso!
* [Practice to perfect](https://vimeo.com/906514592/1ae3a5de05?share=copy)

# ****Preguntas****



*   
*   